#Reverse Mode differentiation of the Cholesky Algorithm in (C|P)ython

[Positive definite matrices](https://en.wikipedia.org/wiki/Positive-definite_matrix) are at the core of Gaussian process models, and are best dealt with using the [Cholesky decomposition](https://en.wikipedia.org/?title=Cholesky_decomposition), which factors a positive definite matrix $\mathbf K$ into a lower triangular matrix $\mathbf L$, as $\mathbf K = \mathbf L\mathbf L^\top$. Once we have $\mathbf L$, it's much easier to solve linear systems, compute determinants, etc. Every inference method in [GPy](https://github.com/SheffieldML/GPy) uses the Cholesky decomposition. 

So let's say we have a matrix $\mathbf K$, which we factor into $\mathbf L$, and then we compute some function $f(\mathbf L)$. For optimization of the function, we compute the derivative $\frac{\partial f}{\partial \mathbf L}$. But since $\mathbf K$ is the primary object, we want to compute $\frac{\partial f}{\partial \mathbf K}$: this is called reverse-mode differentiation (or 'backpropagation' in machine learning parlance), and we'd usually write something like
$$
\frac{\partial f}{\partial \mathbf K} = \sum_{i,j} \frac{\partial f}{\partial \mathbf L}_{[i,j]} \frac{\partial \mathbf L_{[i,j]}}{\partial \mathbf K}
$$

The problem is that $\frac{\partial \mathbf L_{[i,j]}}{\partial \mathbf K}$ is not well defined, because there are _many_ solutions to the equation $\mathbf K = \mathbf L\mathbf L^\top$, of which the Cholesky decomposition picks only one. It turns out that the Cholesky procedure requires a series of square-roots, and by convention the positive root is selected each time: this makes it tricky to continue mathematically, but [Smith](1) shows that by considering the algorithm as a mathematical procedure, we can compute the derivative of each step and end up with the result we want. Adapting Smith's pseudocode to python results in the following.




In [35]:
def backprop_gradient_pure(df_dL, L):
    """
    Given the derivative of an objective fn with respect to the cholesky L,
    compute the derivate with respect to the original matrix K, defined as

        K = LL^T

    where L was obtained by Cholesky decomposition
    """
    df_dK = np.tril(df_dL).copy()
    N = L.shape[0]
    for k in xrange(N - 1, -1, -1):
        for j in xrange(k + 1, N):
            for i in xrange(j, N):
                df_dK[i, k] -= df_dK[i, j] * L[j, k]
                df_dK[j, k] -= df_dK[i, j] * L[i, k]
        for j in xrange(k + 1, N):
            df_dK[j, k] /= L[k, k]
            df_dK[k, k] -= L[j, k] * df_dK[j, k]
        df_dK[k, k] /= (2 * L[k, k])
    return df_dK

Let's see how this works, and how long it takes, for a few different sizes of $\mathbf K$

In [19]:
import numpy as np
for N in [50, 200, 500]:
    #create a P.D. matrix
    A = np.random.randn(N,N+1)
    K = np.dot(A, A.T)

    L = np.linalg.cholesky(K)

    #create a super simple function of L, and its derivative
    B = np.random.randn(*L.shape)
    f = np.sum(L*B)
    df_dL = B
    
    print 'N=', N, ':'
    %timeit backprop_gradient_pure(df_dL, L)
    print ''

100 loops, best of 3: 18.7 ms per loop
1 loops, best of 3: 1.15 s per loop
1 loops, best of 3: 18.3 s per loop


This works, but is horribly slow: the majority of time is spent in Python lookups, garbage collection, and other high level stuff. 

One way round this is to use Cython, which allows us to eliminate that overhead.

In [3]:
%load_ext Cython

In [34]:
%%cython
#cython: wraparaound=False
#cython: boundscheck=False
#cython: nonecheck=False
import numpy as np
cimport numpy as np
def backprop_gradient_cython(np.ndarray[double, ndim=2] df_dL, np.ndarray[double, ndim=2] L):
    cdef np.ndarray[double, ndim=2] df_dK = np.tril(df_dL).copy()
    cdef int N = L.shape[0]
    cdef int k, j, i
    for k in range(N - 1, -1, -1):
        for j in range(k + 1, N):
            for i in range(j, N):
                df_dK[i, k] -= df_dK[i, j] * L[j, k]
                df_dK[j, k] -= df_dK[i, j] * L[i, k]
        for j in range(k + 1, N):
            df_dK[j, k] /= L[k, k]
            df_dK[k, k] -= L[j, k] * df_dK[j, k]
        df_dK[k, k] /= (2. * L[k, k])
    return df_dK

In [30]:
for N in [50, 200, 500, 1000]:
    #create a P.D. matrix
    A = np.random.randn(N,N+1)
    K = np.dot(A, A.T)

    L = np.linalg.cholesky(K)

    #create a super simple function of L, and its derivative
    B = np.random.randn(*L.shape)
    f = np.sum(L*B)
    df_dL = B
    
    print 'N=', N, ':'
    %timeit backprop_gradient_cython(df_dL, L)
    print ''

N= 50 :
10000 loops, best of 3: 82.8 µs per loop

N= 200 :
100 loops, best of 3: 4.01 ms per loop

N= 500 :
10 loops, best of 3: 69.8 ms per loop

N= 1000 :
1 loops, best of 3: 532 ms per loop




    [1]: http://www.tandfonline.com/doi/abs/10.1080/10618600.1995.10474671

With very little effort, cython has given us a much needed improvement. for a 500 by 500 matrix, the backpropagation procedure has gone from 18 seconds to 70 milliseconds, and we're now able to do a 1000 by 1000 matrix in about half a second.

This Cython implementation is still _much_ slower than the decomposition itself though. A thought-provoking question about performance on [Stack Overflow](http://stackoverflow.com/questions/22479258/cholesky-decomposition-with-openmp) shows that the inner loops of the _decomposition_ can be parallelized. Closer inspection of the code above shows that the inner loops of the backpropagation can also be parallelized! 

to do this, We've use the cython parallel code, and have switched the type declarations from numpy arrays to [memoryviews](http://docs.cython.org/src/userguide/memoryviews.html).

In [2]:
%%cython
#cython: wraparaound=False
#cython: boundscheck=False
#cython: nonecheck=False
import numpy as np
cimport numpy as np
from cython.parallel import prange, parallel
def backprop_gradient_cython_par(double[:,:] df_dL, double[:,:] L):
    cdef double[:,:] df_dK = np.tril(df_dL).copy()
    cdef int N = L.shape[0]
    cdef int k, j, i
    for k in range(N - 1, -1, -1):
        with nogil, parallel():
            for i in prange(k + 1, N):
                for j in range(k+1, i+1):
                    df_dK[i, k] -= df_dK[i, j] * L[j, k]
                for j in range(i, N):
                    df_dK[i, k] -= df_dK[j, i] * L[j, k]
        for j in range(k + 1, N):
            df_dK[j, k] /= L[k, k]
            df_dK[k, k] -= L[j, k] * df_dK[j, k]
        df_dK[k, k] /= (2. * L[k, k])
    return df_dK

ERROR: Cell magic `%%cython` not found.


In [42]:
for N in [50, 200, 500, 1000]:
    #create a P.D. matrix
    A = np.random.randn(N,N+1)
    K = np.dot(A, A.T)

    L = np.linalg.cholesky(K)

    #create a super simple function of L, and its derivative
    B = np.random.randn(*L.shape)
    f = np.sum(L*B)
    df_dL = B
    
    print 'N=', N, ':'
    %timeit backprop_gradient_cython_par(df_dL, L)
    print ''

N= 50 :
10000 loops, best of 3: 96.7 µs per loop

N= 200 :
100 loops, best of 3: 3.86 ms per loop

N= 500 :
10 loops, best of 3: 68 ms per loop

N= 1000 :
1 loops, best of 3: 494 ms per loop



I don;t seem to be getting much speed up there. Looking at htop, it seems I'm only getting parallelism for a small amount of time, most of the process is single thread bound. I'm possibly doing something wrong, but it's not clear to me what that is.

One nice thing that Cython allows is for inclusion of functions defined in external c files. This means I can implement the parallelism using `#pragma omp` statements myself. Here's the cython for linking in an externally defined function:

In [16]:
#make sure cython can see the external file at compile time (only needed in the notebook)
import sys
import os
sys.path.append(os.getcwd())

In [24]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --compile-args=-I/home/james/work/WalkingRandomly/
#cython: wraparaound=False
#cython: boundscheck=False
#cython: nonecheck=False
import numpy as np
cimport numpy as np

cdef extern from "cholesky_backprop.h" nogil:
    void chol_backprop(int N, double* df_dK, double* L)

def backprop_gradient_c(np.ndarray[double, ndim=2] df_dL, np.ndarray[double, ndim=2] L):
    cdef np.ndarray[double, ndim=2] dL_dK = np.tril(df_dL) # makes a copy, c-contig
    cdef int N = L.shape[0]
    with nogil:
        chol_backprop(N, <double*> dL_dK.data, <double*> L.data)
    return dL_dK

ImportError: /home/james/.cache/ipython/cython/_cython_magic_fd27ae7de617d79c9185fd6dbcfb7786.so: undefined symbol: chol_backprop

In [7]:
import sys
import os

In [8]:
sys.path.append(os.curdir)

In [10]:
os.curdir

'.'